### Adjust Word Group In Youtube Sentence Word Usage Analysis

This notebook is used to limit the sentence and sentence groups obtained in the previous notebook (2-Word Group In Youtube Rev2.ipynb) according to word usage. (Analysis1)

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize 
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [6]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [9]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,gewoon niet meer weten maar wat is dan nog ja ...,1715,1724,gewoon niet meer weten maar wat is dan nog ja ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
1,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...
2,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
3,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
4,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
...,...,...,...,...,...,...
119043,en of je nu,92,93,en of je nu jong of ouder bent iedereen voelt ...,P21plwYDGuE,NaN
119044,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,NaN
119045,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,NaN
119046,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,NaN


In [10]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,gewoon niet meer weten maar wat is dan nog ja ...,1715,1724,gewoon niet meer weten maar wat is dan nog ja ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[121, 7, 79, 117, 18, 10, 5, 35, 42, 48, 18, 4..."
1,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."
2,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
3,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
4,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
...,...,...,...,...,...,...,...
119043,en of je nu,92,93,en of je nu jong of ouder bent iedereen voelt ...,P21plwYDGuE,NaN,"[8, 65, 1, 54]"
119044,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,NaN,"[0, 60, 33, 3]"
119045,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,NaN,"[33, 4, 33, 3]"
119046,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,NaN,"[0, 60, 33, 12]"


In [11]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [12]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
...,...,...,...,...,...,...,...
65970,en of je nu,92,93,en of je nu jong of ouder bent iedereen voelt ...,P21plwYDGuE,NaN,"[8, 65, 1, 54]"
65971,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,NaN,"[0, 60, 33, 3]"
65972,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,NaN,"[33, 4, 33, 3]"
65973,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,NaN,"[0, 60, 33, 12]"


In [13]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [14]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [15]:
#word_num_index_dict

In [16]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [17]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[139,
 140,
 147,
 150,
 152,
 26,
 155,
 156,
 158,
 160,
 163,
 169,
 170,
 175,
 176,
 177,
 179,
 184,
 189,
 190,
 192,
 195,
 196,
 71,
 72,
 199,
 77,
 94,
 98,
 110,
 116,
 124]

In [18]:
#Counter(index_list)

In [19]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
...,...,...,...,...,...,...,...
94,nee denk eens goed,475,478,dat ik oude oma nee denk eens goed beleid van ...,ORrmZ6Na6vk,NaN,"[51, 90, 99, 50]"
95,was uw vader was,15,17,sabena was uw vader was zelfs nog voogdijminis...,GcLMEXhvuy0,NaN,"[27, 136, 151, 27]"
96,doe mee denk mee,10,12,doe mee denk mee,it4aVZFumUo,NaN,"[87, 105, 90, 105]"
97,nee nee nee nee,385,387,maar je bedoelde nee nee nee nee vanuit het so...,OrphmzMjQc4,NaN,"[51, 51, 51, 51]"


In [20]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
...,...,...,...,...,...,...,...
94,nee denk eens goed,475,478,dat ik oude oma nee denk eens goed beleid van ...,ORrmZ6Na6vk,https://www.youtube.com/watch?v=ORrmZ6Na6vk&t=...,"[51, 90, 99, 50]"
95,was uw vader was,15,17,sabena was uw vader was zelfs nog voogdijminis...,GcLMEXhvuy0,https://www.youtube.com/watch?v=GcLMEXhvuy0&t=15s,"[27, 136, 151, 27]"
96,doe mee denk mee,10,12,doe mee denk mee,it4aVZFumUo,https://www.youtube.com/watch?v=it4aVZFumUo&t=10s,"[87, 105, 90, 105]"
97,nee nee nee nee,385,387,maar je bedoelde nee nee nee nee vanuit het so...,OrphmzMjQc4,https://www.youtube.com/watch?v=OrphmzMjQc4&t=...,"[51, 51, 51, 51]"


In [21]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [22]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
...,...,...,...,...,...,...,...
94,nee denk eens goed,475,478,dat ik oude oma nee denk eens goed beleid van ...,ORrmZ6Na6vk,https://www.youtube.com/watch?v=ORrmZ6Na6vk&t=...,"[51, 90, 99, 50]"
95,was uw vader was,15,17,sabena was uw vader was zelfs nog voogdijminis...,GcLMEXhvuy0,https://www.youtube.com/watch?v=GcLMEXhvuy0&t=15s,"[27, 136, 151, 27]"
96,doe mee denk mee,10,12,doe mee denk mee,it4aVZFumUo,https://www.youtube.com/watch?v=it4aVZFumUo&t=10s,"[87, 105, 90, 105]"
97,nee nee nee nee,385,387,maar je bedoelde nee nee nee nee vanuit het so...,OrphmzMjQc4,https://www.youtube.com/watch?v=OrphmzMjQc4&t=...,"[51, 51, 51, 51]"


In [23]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [24]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,aan,7
1,kan,7
2,naar,7
3,misschien,7
4,mijn,7
...,...,...
163,staat,1
164,dacht,1
165,tijd,1
166,uw,1


In [25]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [26]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [27]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [28]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [29]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['z',
 'h',
 'jouw',
 'jaar',
 'dag',
 'n',
 'wacht',
 'zie',
 'kunt',
 'geef',
 'vrouw',
 'mr',
 'bedankt',
 'waren',
 'spijt',
 'niemand',
 'werk',
 'niks',
 'moeder',
 'zullen',
 'na',
 'genoeg',
 'wist',
 'm',
 'kom',
 'neem',
 't',
 's',
 'ok',
 'komt',
 'toen',
 'twee']

In [30]:
set_not_in_sent_word = set(not_in_sent_word)

In [31]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [32]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'bedankt',
 'dag',
 'geef',
 'genoeg',
 'h',
 'jaar',
 'jouw',
 'kom',
 'komt',
 'kunt',
 'm',
 'moeder',
 'mr',
 'n',
 'na',
 'neem',
 'niemand',
 'niks',
 'ok',
 's',
 'spijt',
 't',
 'toen',
 'twee',
 'vrouw',
 'wacht',
 'waren',
 'werk',
 'wist',
 'z',
 'zie',
 'zullen'}

In [33]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['t',
 'kunt',
 'vrouw',
 'geef',
 'jaar',
 'na',
 'toen',
 'spijt',
 'waren',
 'zie',
 'mr',
 'twee',
 'neem',
 'h',
 'zullen',
 'moeder',
 'kom',
 's',
 'wist',
 'werk',
 'niemand',
 'dag',
 'bedankt',
 'jouw',
 'wacht',
 'z',
 'ok',
 'niks',
 'm',
 'komt',
 'genoeg',
 'n']

In [34]:
len(not_in_sent_selected_word_list)

32

In [35]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
...,...,...,...,...,...,...
65970,en of je nu,92,93,en of je nu jong of ouder bent iedereen voelt ...,P21plwYDGuE,NaN
65971,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,NaN
65972,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,NaN
65973,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,NaN


In [36]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [37]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [38]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,t,toen zei mijn vrouw moet je t gewoon,634,636,toen zei mijn vrouw moet je t gewoon zelf gaan...,robgEk58nx4,https://www.youtube.com/watch?v=robgEk58nx4&t=...
1,t,van de t nu en dan zie je gewoon het,551,554,de hoge school met deels ik ook groot deel van...,xKNDkrAQ3iM,https://www.youtube.com/watch?v=xKNDkrAQ3iM&t=...
2,t,alleen ja ik weet dus niet wie het t,200,204,alleen ja ik weet dus niet wie het t beste kan,q3i547Tf13Y,https://www.youtube.com/watch?v=q3i547Tf13Y&t=...
3,t,al ik wil niet ik wil niet t dit,323,326,heb mijn bedrijf al ik wil niet ik wil niet t ...,GHb27jxODNA,https://www.youtube.com/watch?v=GHb27jxODNA&t=...
4,t,zijn omdat ze die t er zou een,624,626,mensen zijn die bond een gebied heel goed kenn...,IAyQeJ57A0s,https://www.youtube.com/watch?v=IAyQeJ57A0s&t=...
...,...,...,...,...,...,...,...
147,n,wat moet je doen in zo n tijd je kan niks doen,239,244,wat moet je doen in zo n tijd je kan niks doen,rYDe9r2Tkao,https://www.youtube.com/watch?v=rYDe9r2Tkao&t=...
148,n,zijn dus voor mij was het n,507,509,feit dat wauw er zijn zoveel mensen hier die b...,GHb27jxODNA,https://www.youtube.com/watch?v=GHb27jxODNA&t=...
149,n,n gaan doe ja ik ben heel,481,484,waaro je u dat bet n gaan doe ja ik ben heel a...,sYCGipdczVc,https://www.youtube.com/watch?v=sYCGipdczVc&t=...
150,n,dan n gaan ze zich op een,521,523,zo intelligent dat ze als één ding uitval dan ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...


In [39]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [40]:
not_in_sent_selected_word_list_result.word.nunique()

32

In [41]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [42]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [43]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [44]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,t,toen zei mijn vrouw moet je t gewoon,634,636,toen zei mijn vrouw moet je t gewoon zelf gaan...,robgEk58nx4,https://www.youtube.com/watch?v=robgEk58nx4&t=...
1,t,van de t nu en dan zie je gewoon het,551,554,de hoge school met deels ik ook groot deel van...,xKNDkrAQ3iM,https://www.youtube.com/watch?v=xKNDkrAQ3iM&t=...
2,t,alleen ja ik weet dus niet wie het t,200,204,alleen ja ik weet dus niet wie het t beste kan,q3i547Tf13Y,https://www.youtube.com/watch?v=q3i547Tf13Y&t=...
3,t,al ik wil niet ik wil niet t dit,323,326,heb mijn bedrijf al ik wil niet ik wil niet t ...,GHb27jxODNA,https://www.youtube.com/watch?v=GHb27jxODNA&t=...
4,t,zijn omdat ze die t er zou een,624,626,mensen zijn die bond een gebied heel goed kenn...,IAyQeJ57A0s,https://www.youtube.com/watch?v=IAyQeJ57A0s&t=...
...,...,...,...,...,...,...,...
147,n,wat moet je doen in zo n tijd je kan niks doen,239,244,wat moet je doen in zo n tijd je kan niks doen,rYDe9r2Tkao,https://www.youtube.com/watch?v=rYDe9r2Tkao&t=...
148,n,zijn dus voor mij was het n,507,509,feit dat wauw er zijn zoveel mensen hier die b...,GHb27jxODNA,https://www.youtube.com/watch?v=GHb27jxODNA&t=...
149,n,n gaan doe ja ik ben heel,481,484,waaro je u dat bet n gaan doe ja ik ben heel a...,sYCGipdczVc,https://www.youtube.com/watch?v=sYCGipdczVc&t=...
150,n,dan n gaan ze zich op een,521,523,zo intelligent dat ze als één ding uitval dan ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...


In [45]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [46]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx']

In [47]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [48]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass